In [1]:
from langchain_ollama import ChatOllama


llm = ChatOllama(model="hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:F16")

In [2]:


llm.invoke(input="""
You are an expert in PostgreSQL. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of PostgreSQL to generate an SQL statement that answers the [User Question].

[User Question]
Find the names of authors who have written more than 5 papers.

[Database Schema]
CREATE TABLE author (
  authorid INT,
  authorname VARCHAR(255)
);
CREATE TABLE writes (
  paperid INT,
  authorid INT
);

[Reference Information]
None

[User Question]
Find the names of authors who have written more than 5 papers.

```sql""")

AIMessage(content='SELECT a.authorname FROM author a JOIN writes w ON a.authorid = w.authorid GROUP BY a.authorid HAVING COUNT(w.paperid) > 5;', additional_kwargs={}, response_metadata={'model': 'hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:F16', 'created_at': '2025-06-19T02:30:02.020707Z', 'done': True, 'done_reason': 'stop', 'total_duration': 51464167794, 'load_duration': 18583446882, 'prompt_eval_count': 138, 'prompt_eval_duration': 18115351550, 'eval_count': 35, 'eval_duration': 14752386873, 'model_name': 'hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:F16'}, id='run--a1f47605-9893-42f6-aeb8-7607e0979c33-0', usage_metadata={'input_tokens': 138, 'output_tokens': 35, 'total_tokens': 173})

In [3]:
for step in llm.stream(
    input="""
You are an expert in PostgreSQL. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of PostgreSQL to generate an SQL statement that answers the [User Question].

[User Question]
Find the names of authors who have written more than 5 papers.

[Database Schema]
CREATE TABLE author (
  authorid INT,
  authorname VARCHAR(255)
);
CREATE TABLE writes (
  paperid INT,
  authorid INT
);

[Reference Information]
None

[User Question]
Find the names of authors who have written more than 5 papers.
```sql"""
):
    print(step.text(), end="")

SELECT a.authorname FROM author a JOIN writes w ON a.authorid = w.authorid GROUP BY a.authorid HAVING COUNT(w.paperid) > 5;

In [4]:
def stream(prompt: str):
    for step in llm.stream(prompt):
        print(step.text(), end="")

In [5]:
stream(
    "You are an expert in PostgreSQL. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of PostgreSQL to generate an SQL statement that answers the [User Question]."
)

SELECT COUNT(*) FROM "public"."order" WHERE "status" = 'cancelled';

In [6]:
stream(
    """
[User Question]
Find the names of authors who have written more than 5 papers.

[Database Schema]
CREATE TABLE author (
  authorid INT,
  authorname VARCHAR(255)
);
CREATE TABLE writes (
  paperid INT,
  authorid INT
);

[Reference Information]
None

[User Question]
Find the names of authors who have written more than 5 papers.
```sql
       """
)

SELECT T1.authorname FROM author AS T1 JOIN writes AS T2 ON T1.authorid = T2.authorid GROUP BY T2.authorid HAVING COUNT(*) > 5

In [7]:
stream("HELLO")

Hello! How can I assist you today?

In [9]:
stream(
    "System: You are an expert in PostgreSQL. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of PostgreSQL to generate an SQL statement that answers the [User Question]. Now I will provide you the question and you need to answer that."
)

Understood! Please go ahead and ask your question, and I'll generate the appropriate SQL statement for you based on the provided database schema and reference information.

In [10]:
stream(
    """
[User Question]
Find the names of authors who have written more than 5 papers.

[Database Schema]
CREATE TABLE author (
  authorid INT,
  authorname VARCHAR(255)
);
CREATE TABLE writes (
  paperid INT,
  authorid INT
);

[Reference Information]
None

[User Question]
Find the names of authors who have written more than 5 papers.
```sql
       """
)

SELECT T1.authorname FROM author AS T1 JOIN writes AS T2 ON T1.authorid = T2.authorid GROUP BY T2.authorid HAVING COUNT(*) > 5

In [15]:
stream(
    """
Here is the database information
Overview:
Chinook is a fictional digital media store database designed for learning and demos. It models real-world entities—artists, albums, tracks, customers, invoices, employees, playlists, and more—using data inspired by an iTunes library and synthetic sales information. The schema includes 11 relational tables with realistic constraints, indexes, and over 15,000 rows of data.

Tables & Descriptions:
employees: Information on store staff, including personal details and reporting hierarchy; used for assigning customer support roles and analyzing performance.
customers: Stores customer profiles and contacts, linking them to their support representatives; essential for order history and marketing.
invoices: Invoice headers containing transaction metadata like date, billing address, and total; core for revenue tracking and financial analysis.
invoice_items (aka invoice_line): Line-item details for each invoice (track sold with price and qty); crucial for item-level sales reporting.
artists: Catalog of music artists; foundational for grouping albums and tracks.
albums: Albums tied to artists; used for browsing catalog by album and aggregated media views.
media_types: Defines format types (e.g., MP3, AAC); supports format-based filtering or reporting.
genres: Standard music genres (Rock, Jazz, etc.); enables genre-based analytics and recommendations.
tracks: Core catalog of songs with metadata (title, composer, length, price); central to browsing and sales.
playlists: Named user or system playlists; used for organizing and retrieving track collections.
playlist_track: Many-to-many join between playlists and tracks; supports playlist management and content queries.
    
    Your task is choosing the tables can be used to answer the user question
    User question: Which composers's tracks have generated over $100 in total sales?
    
    ONLY OUTPUT THE CHOSEN TABLES in a comma seperated list
    """
)

tracks, invoice_items

In [23]:
stream(
    """
  You are an expert in PostgreSQL. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of PostgreSQL to generate an SQL statement that answers the [User Question].

[User Question]
Which composers's tracks have generated over $100 in total sales?

[Database Schema]
CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")
);
CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

[Reference Information]
3 rows from InvoiceLine table:
InvoiceLineId	InvoiceId	TrackId	UnitPrice	Quantity
1	1	2	0.99	1
2	1	4	0.99	1
3	2	6	0.99	1
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Angus Young, Malcolm Young, Brian Johnson	343719	11170334	0.99
2	Balls to the Wall	2	2	1	U. Dirkschneider, W. Hoffmann, H. Frank, P. Baltes, S. Kaufmann, G. Hoffmann	342562	5510424	0.99
3	Fast As a Shark	3	2	1	F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman	230619	3990994	0.99

[User Question]
Which composers's tracks have generated over $100 in total sales?
```sql
  """
)

SELECT t.Composer FROM Track t JOIN InvoiceLine il ON t.TrackId = il.TrackId GROUP BY t.Composer HAVING SUM(il.UnitPrice * il.Quantity) > 100;

In [19]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../../Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.get_table_info(["Track", "InvoiceLine"])

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


'\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/\n\n\nCREATE TABLE "Track" (\n\t"TrackId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(200) NOT NULL, \n\t"AlbumId" INTEGER, \n\t"MediaTypeId" INTEGER NOT NULL, \n\t"GenreId" INTEGER, \n\t"Composer" NVARCHAR(220), \n\t"Milliseconds" INTEGER NOT NULL, \n\t"Bytes" INTEGER, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\tPRIMARY KEY ("TrackId"), \n\tFOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), \n\tFOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), \n\tFOREIGN KE

In [20]:
print(
    '\nCREATE TABLE "InvoiceLine" (\n\t"InvoiceLineId" INTEGER NOT NULL, \n\t"InvoiceId" INTEGER NOT NULL, \n\t"TrackId" INTEGER NOT NULL, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\t"Quantity" INTEGER NOT NULL, \n\tPRIMARY KEY ("InvoiceLineId"), \n\tFOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), \n\tFOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")\n)\n\n/*\n3 rows from InvoiceLine table:\nInvoiceLineId\tInvoiceId\tTrackId\tUnitPrice\tQuantity\n1\t1\t2\t0.99\t1\n2\t1\t4\t0.99\t1\n3\t2\t6\t0.99\t1\n*/\n\n\nCREATE TABLE "Track" (\n\t"TrackId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(200) NOT NULL, \n\t"AlbumId" INTEGER, \n\t"MediaTypeId" INTEGER NOT NULL, \n\t"GenreId" INTEGER, \n\t"Composer" NVARCHAR(220), \n\t"Milliseconds" INTEGER NOT NULL, \n\t"Bytes" INTEGER, \n\t"UnitPrice" NUMERIC(10, 2) NOT NULL, \n\tPRIMARY KEY ("TrackId"), \n\tFOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), \n\tFOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), \n\tFOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")\n)\n\n/*\n3 rows from Track table:\nTrackId\tName\tAlbumId\tMediaTypeId\tGenreId\tComposer\tMilliseconds\tBytes\tUnitPrice\n1\tFor Those About To Rock (We Salute You)\t1\t1\t1\tAngus Young, Malcolm Young, Brian Johnson\t343719\t11170334\t0.99\n2\tBalls to the Wall\t2\t2\t1\tU. Dirkschneider, W. Hoffmann, H. Frank, P. Baltes, S. Kaufmann, G. Hoffmann\t342562\t5510424\t0.99\n3\tFast As a Shark\t3\t2\t1\tF. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman\t230619\t3990994\t0.99\n*/'
)


CREATE TABLE "InvoiceLine" (
	"InvoiceLineId" INTEGER NOT NULL, 
	"InvoiceId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	"Quantity" INTEGER NOT NULL, 
	PRIMARY KEY ("InvoiceLineId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("InvoiceId") REFERENCES "Invoice" ("InvoiceId")
)

/*
3 rows from InvoiceLine table:
InvoiceLineId	InvoiceId	TrackId	UnitPrice	Quantity
1	1	2	0.99	1
2	1	4	0.99	1
3	2	6	0.99	1
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track tabl